In [6]:
import json
from tqdm import tqdm
import csv

def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

file_path = "/Users/hsinhochien/Documents/college/大三下/NLP/fp_2.2/Task 2.2/train.json"
train_data = load_json(file_path)

file_path = "/Users/hsinhochien/Documents/college/大三下/NLP/fp_2.2/Task 2.2/dev.json"
val_data = load_json(file_path)

In [3]:
# Import packages
import google.generativeai as genai
from typing import List, Tuple
import json

# Set up Gemini API key
## TODO: Fill in your Gemini API in the ""
GOOGLE_API_KEY=""
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# Check if you have set your Gemini API successfully
# You should see "Set Gemini API sucessfully!!" if nothing goes wrong.
try:
    model.generate_content(
      "test",
    )
    print("Set Gemini API sucessfully!!")
except:
    print("There seems to be something wrong with your Gemini API. Please follow our demonstration in the slide to get a correct one.")

Set Gemini API sucessfully!!


In [11]:
prompt_1 = f"Given two sentences, classify the relation between them into one of the following classes:\n"\
            "Class 0: There is no detected relation between the two sentences.\n"\
            "Class 1: There is a 'Support' relation from sentence 1 to sentence 2.\n"\
            "Class 2: There is an 'Attack' relation from sentence 1 to sentence 2.\n"\
            "Sentence 1: '{}'\n"\
            "Sentence 2: '{}'\n"\
            "Classify the relation between the two sentences:"

prompt_2 = f"Given two sentences, classify the relation between them into one of the following classes:\n"\
            "Class 0: There is no detected relation between the two sentences.\n"\
            "Class 1: There is a 'Support' relation from sentence 1 to sentence 2.\n"\
            "        Explanation: Sentence 1 provides evidence, justification, or reinforcement for sentence 2.\n"\
            "Class 2: There is an 'Attack' relation from sentence 1 to sentence 2.\n"\
            "        Explanation: Sentence 1 undermines, contradicts, or challenges the validity of sentence 2.\n"\
            "Sentence 1: '{}'\n"\
            "Sentence 2: '{}'\n"\
            "Please carefully analyze the semantic relationship between the two sentences before making a classification.\n"\
            "Remember to consider both the explicit and implicit meanings of the sentences.\n"\
            "Example Usage: If the two sentences are:\n"\
            "Sentence 1: 'Japan as a geography for us is a high transactional market.'\n"\
            "Sentence 2: 'The improvement in that in Q3 is obviously very high margin and also the bottom.'\n"\
            "then the relationship between them is support."

# Define the function to classify the relation between two sentences
def classify_relation(p, sentence_1, sentence_2):
    prompt = p.format(sentence_1, sentence_2)
    response = model.generate_content(prompt)
    # Check if response contains a valid Part
    if response.parts:
        raw_response = response.parts[0].text
    else:
        # If response is blocked, return None
        raw_response = None

    if raw_response:
        # Extract the predicted class from the response
        parts = raw_response.split(":")
        class_value = parts[0].strip().split()[1]
        class_value_str = str(class_value)
        predicted_class = int(class_value_str.split('.')[0])
    else:
        predicted_class = None

    return raw_response, predicted_class

def calculate_accuracy(predicted_classes, actual_classes):
    correct_predictions = sum(1 for pred, actual in zip(predicted_classes, actual_classes) if pred == actual)
    total_predictions = len(actual_classes)
    accuracy = correct_predictions / total_predictions
    return accuracy

### prompt 1

In [46]:
actual_classes = []
raw_responses = []
predicted_classes = []

for sentence_pair in tqdm(val_data):
    sentence_1, sentence_2, label = sentence_pair
    actual_classes.append(label)
    raw_response, predicted_class = classify_relation(prompt_1, sentence_1, sentence_2)
    raw_responses.append(raw_response)
    predicted_classes.append(predicted_class)

100%|██████████| 690/690 [13:26<00:00,  1.17s/it]


In [47]:
# Save raw responses to a JSON file
with open("raw_responses_1.json", "w") as json_file:
    json.dump(raw_responses, json_file)

print(f"Predictions saved to raw_responses_1.json")

# Save predicted classes to a CSV file
with open("predicted_classes_1.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Prediction", "True Label"])
    writer.writerows(zip(predicted_classes, actual_classes))

print(f"Predictions saved to predicted_classes_1.json")

Predictions saved to raw_responses_1.json
Predictions saved to predicted_classes_1.json


In [48]:
accuracy = calculate_accuracy(predicted_classes, actual_classes)
print("Accuracy for prompt 1:", accuracy)

Accuracy for prompt 1: 0.5057971014492754


### prompt 2

In [12]:
actual_classes = []
raw_responses = []
predicted_classes = []

for sentence_pair in tqdm(val_data):
    sentence_1, sentence_2, label = sentence_pair
    actual_classes.append(label)
    raw_response, predicted_class = classify_relation(prompt_2, sentence_1, sentence_2)
    raw_responses.append(raw_response)
    predicted_classes.append(predicted_class)

100%|██████████| 690/690 [16:21<00:00,  1.42s/it]


In [13]:
# Save raw responses to a JSON file
with open("raw_responses_2.json", "w") as json_file:
    json.dump(raw_responses, json_file)

print(f"Predictions saved to raw_responses_2.json")

# Save predicted classes to a CSV file
with open("predicted_classes_2.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Prediction", "True Label"])
    writer.writerows(zip(predicted_classes, actual_classes))

print(f"Predictions saved to predicted_classes_2.json")

Predictions saved to raw_responses_2.json
Predictions saved to predicted_classes_2.json


In [14]:
accuracy = calculate_accuracy(predicted_classes, actual_classes)
print("Accuracy for prompt 2:", accuracy)

Accuracy for prompt 2: 0.5681159420289855
